<a href="https://colab.research.google.com/github/Dhanushsirigineedi/DL_Assignment_3/blob/main/Assignment_3_without_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq Model for Dakshina Transliteration

This notebook implements a sequence-to-sequence model for transliteration using the Dakshina dataset.

## 1. Installation and Setup

In [1]:
# Install required packages
!pip install wandb

In [2]:
# Import necessary libraries
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import copy
from torch.utils.data import Dataset, DataLoader
import random
import os
import pickle
import subprocess
import sys
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
import wandb
import json
import matplotlib.pyplot as plt
import numpy as np
import types

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2. Vocabulary and Dataset Classes

In [3]:
# Vocabulary class
class CharVocab:
    """Character-level vocabulary class with special tokens support"""
    def __init__(self, tokens=None, specials=['<pad>','<sos>','<eos>','<unk>']):
        self.specials = specials
        self.idx2char = list(specials) + (tokens or [])
        self.char2idx = {ch:i for i,ch in enumerate(self.idx2char)}

    @classmethod
    def build_from_texts(cls, texts):
        """Build vocabulary from a list of texts"""
        chars = sorted({c for line in texts for c in line})
        return cls(tokens=chars)

    @classmethod
    def build_from_file(cls, file_path, src_col=0, tgt_col=1, is_csv=True):
        """Build vocabulary from a data file (CSV or TSV)"""
        if is_csv:
            df = pd.read_csv(file_path, header=None)
            texts = df[src_col].dropna().tolist() + df[tgt_col].dropna().tolist()
        else:
            texts = []
            with open(file_path, encoding='utf-8') as f:
                for ln in f:
                    parts = ln.strip().split('\t')
                    if len(parts) >= 2:
                        texts.extend([parts[0], parts[1]])

        return cls.build_from_texts(texts)

    def save(self, path):
        """Save vocabulary to JSON file"""
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.idx2char, f, ensure_ascii=False)

    @classmethod
    def load(cls, path):
        """Load vocabulary from JSON file"""
        with open(path, encoding='utf-8') as f:
            idx2char = json.load(f)

        inst = cls(tokens=[])
        inst.idx2char = idx2char
        inst.char2idx = {c:i for i,c in enumerate(idx2char)}
        return inst

    def encode(self, text, add_sos=False, add_eos=False):
        """Convert text to a sequence of indices"""
        seq = []
        if add_sos: seq.append(self.char2idx['<sos>'])
        for c in text:
            seq.append(self.char2idx.get(c, self.char2idx['<unk>']))
        if add_eos: seq.append(self.char2idx['<eos>'])
        return seq

    def decode(self, idxs, strip_specials=True, join=True):
        """Convert a sequence of indices back to text"""
        # Convert tensor to list if needed
        if hasattr(idxs, 'tolist'):
            idxs = idxs.tolist()

        # Convert indices to characters
        chars = [self.idx2char[i] for i in idxs if i < len(self.idx2char)]

        # Remove special tokens if requested
        if strip_specials:
            chars = [c for c in chars if c not in self.specials]

        # Return as string or list
        return ''.join(chars) if join else chars

    def batch_decode(self, batch_idxs, strip_specials=True):
        """Decode a batch of index sequences"""
        return [self.decode(seq, strip_specials=strip_specials) for seq in batch_idxs]

    def __len__(self):
        return len(self.idx2char)

    @property
    def pad_idx(self): return self.char2idx['<pad>']

    @property
    def sos_idx(self): return self.char2idx['<sos>']

    @property
    def eos_idx(self): return self.char2idx['<eos>']

    @property
    def unk_idx(self): return self.char2idx['<unk>']

    @property
    def size(self): return len(self.idx2char)

In [4]:
# Dataset class
class TransliterationDataset(Dataset):
    """A dataset class for Dakshina-style transliteration data"""

    def __init__(self, path, src_vocab, tgt_vocab, format='dakshina'):
        """Initialize the dataset"""
        self.examples = []
        self.format = format

        if format == 'dakshina':
            # Dakshina format: tab-separated without header
            with open(path, encoding='utf-8') as f:
                for ln in f:
                    parts = ln.strip().split('\t')
                    if len(parts) >= 2:
                        # In Dakshina, the format is target, source (native, latin)
                        tgt, src = parts[0], parts[1]
                        src_ids = src_vocab.encode(src, add_sos=True, add_eos=True)
                        tgt_ids = tgt_vocab.encode(tgt, add_sos=True, add_eos=True)
                        self.examples.append((
                            torch.tensor(src_ids, dtype=torch.long),
                            torch.tensor(tgt_ids, dtype=torch.long)
                        ))
        else:
            # CSV format
            df = pd.read_csv(path, header=None)
            for _, row in df.iterrows():
                src = row[0]
                tgt = row[1]
                src_ids = src_vocab.encode(src, add_sos=True, add_eos=True)
                tgt_ids = tgt_vocab.encode(tgt, add_sos=True, add_eos=True)
                self.examples.append((
                    torch.tensor(src_ids, dtype=torch.long),
                    torch.tensor(tgt_ids, dtype=torch.long)
                ))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

## 3. Dataset Downloading and Processing

In [5]:
# Dataset download and processing
def download_dakshina_dataset(base_dir=None):
    """Download and extract the Dakshina dataset"""
    if base_dir is None:
        base_dir = './dakshina'

    os.makedirs(base_dir, exist_ok=True)

    # File paths
    tar_path = os.path.join(base_dir, 'dakshina_dataset_v1.0.tar')
    extract_path = os.path.join(base_dir, 'dakshina_dataset_v1.0')

    # Check if the dataset is already downloaded and extracted
    if os.path.exists(extract_path):
        print(f"Dakshina dataset already exists at {extract_path}")
        return extract_path

    # Download the dataset
    print("Downloading Dakshina dataset...")
    url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

    # Use subprocess.run for better compatibility across platforms
    try:
        subprocess.run(['wget', url, '-O', tar_path], check=True)
    except:
        # Fallback for systems without wget
        try:
            import requests
            print("Using requests to download (this may take a while)...")
            response = requests.get(url, stream=True)
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 Kibibyte

            with open(tar_path, 'wb') as f:
                for i, data in enumerate(response.iter_content(block_size)):
                    if i % 1000 == 0:
                        progress = i * block_size / total_size * 100 if total_size > 0 else 0
                        print(f"Downloaded {progress:.1f}%")
                    f.write(data)
        except Exception as e:
            print(f"Error downloading: {e}")
            print("Failed to download the dataset. Please download it manually from:")
            print(url)
            return None

    # Extract the dataset
    print(f"Extracting to {extract_path}...")
    try:
        subprocess.run(['tar', '-xf', tar_path, '-C', base_dir], check=True)
    except Exception as e:
        print(f"Error extracting: {e}")
        print("Failed to extract the dataset. Please extract it manually.")
        return None

    print(f"Dakshina dataset extracted to {extract_path}")
    return extract_path

def collate_fn(batch, src_vocab, tgt_vocab):
    """Collate function to handle variable-length sequences"""
    srcs, tgts = zip(*batch)
    srcs_p = pad_sequence(srcs, batch_first=True, padding_value=src_vocab.pad_idx)
    tgts_p = pad_sequence(tgts, batch_first=True, padding_value=tgt_vocab.pad_idx)
    src_lens = torch.tensor([len(s) for s in srcs], dtype=torch.long)
    return srcs_p, src_lens, tgts_p

def get_dataloaders(
        language='te',
        dataset_format='dakshina',
        base_path=None,
        batch_size=64,
        device='cpu',
        cache_dir='./cache',
        use_cached_vocab=True
    ):
    """Load transliteration datasets for Dakshina"""
    # Create cache directory if it doesn't exist
    if use_cached_vocab:
        os.makedirs(cache_dir, exist_ok=True)
        vocab_cache_path = os.path.join(cache_dir, f"{language}_{dataset_format}_vocab.pkl")

    # Set up paths based on dataset format
    if base_path is None:
        dakshina_path = download_dakshina_dataset()
        if dakshina_path is None:
            # If download failed, try with the default Kaggle path
            dakshina_path = '/kaggle/working/dakshina_dataset_v1.0'

        base_path = os.path.join(
            dakshina_path,
            language, 'lexicons'
        )

    # Try to load cached vocabularies
    if use_cached_vocab and os.path.exists(vocab_cache_path):
        print(f"Loading cached vocabularies from {vocab_cache_path}")
        with open(vocab_cache_path, 'rb') as f:
            src_vocab, tgt_vocab = pickle.load(f)
    else:
        # Build vocabularies from data
        all_src, all_tgt = [], []

        # In Dakshina, files are named like: language.translit.sampled.{split}.tsv
        if dataset_format == 'dakshina':
            for split in ['train', 'dev']:
                path = os.path.join(base_path, f"{language}.translit.sampled.{split}.tsv")
                with open(path, encoding='utf-8') as f:
                    for ln in f:
                        parts = ln.strip().split('\t')
                        if len(parts) >= 2:
                            # Dakshina format: native script (target), latin script (source)
                            all_tgt.append(parts[0])
                            all_src.append(parts[1])
        else:
            # CSV format
            train_df = pd.read_csv(os.path.join(base_path, f"{language}_train.csv"), header=None)
            val_df = pd.read_csv(os.path.join(base_path, f"{language}_valid.csv"), header=None)

            all_src = train_df[0].tolist() + val_df[0].tolist()
            all_tgt = train_df[1].tolist() + val_df[1].tolist()

        # Build vocabularies
        src_vocab = CharVocab.build_from_texts(all_src)
        tgt_vocab = CharVocab.build_from_texts(all_tgt)

        # Cache vocabularies
        if use_cached_vocab:
            with open(vocab_cache_path, 'wb') as f:
                pickle.dump((src_vocab, tgt_vocab), f)

    # Create data loaders for each split
    loaders = {}

    if dataset_format == 'dakshina':
        splits = {'train': 'train', 'val': 'dev', 'test': 'test'}
        for split_name, file_split in splits.items():
            path = os.path.join(base_path, f"{language}.translit.sampled.{file_split}.tsv")
            try:
                ds = TransliterationDataset(path, src_vocab, tgt_vocab, format='dakshina')
                loaders[split_name] = DataLoader(
                    ds,
                    batch_size=batch_size,
                    shuffle=(split_name == 'train'),
                    collate_fn=lambda b: collate_fn(b, src_vocab, tgt_vocab)
                )
            except FileNotFoundError:
                print(f"Warning: File not found: {path}")
    else:
        # CSV format
        splits = {
            'train': f"{language}_train.csv",
            'val': f"{language}_valid.csv",
            'test': f"{language}_test.csv"
        }

        for split_name, file_name in splits.items():
            path = os.path.join(base_path, file_name)
            try:
                ds = TransliterationDataset(path, src_vocab, tgt_vocab, format='csv')
                loaders[split_name] = DataLoader(
                    ds,
                    batch_size=batch_size,
                    shuffle=(split_name == 'train'),
                    collate_fn=lambda b: collate_fn(b, src_vocab, tgt_vocab)
                )
            except FileNotFoundError:
                print(f"Warning: File not found: {path}")

    return loaders, src_vocab, tgt_vocab

## 4. Model Architecture

In [6]:
# Attention mechanism
class BahdanauAttention(nn.Module):
    def __init__(self, enc_hid, dec_hid):
        super().__init__()
        self.attn = nn.Linear(enc_hid + dec_hid, dec_hid)
        self.v = nn.Linear(dec_hid, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask):
        # hidden: [B, H], encoder_outputs: [B, T, H], mask: [B, T]
        B, T, H = encoder_outputs.size()
        hidden = hidden.unsqueeze(1).repeat(1, T, 1)               # [B, T, H]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [B, T, H]
        scores = self.v(energy).squeeze(2)                        # [B, T]
        scores = scores.masked_fill(~mask, -1e9)
        return torch.softmax(scores, dim=1)                       # [B, T]

In [7]:
# Encoder
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hid_size, layers=1, cell='LSTM', dropout=0.0, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.bidirectional = bidirectional
        self.cell_type = cell
        self.layers = layers
        self.hidden_size = hid_size

        # Output size will be doubled if bidirectional
        self.output_size = hid_size * 2 if bidirectional else hid_size

        rnn_cls = {'LSTM': nn.LSTM, 'GRU': nn.GRU, 'RNN': nn.RNN}[cell]
        self.rnn = rnn_cls(emb_size,
                         hid_size,
                         num_layers=layers,
                         dropout=dropout if layers>1 else 0.0,
                         batch_first=True,
                         bidirectional=bidirectional)

    def forward(self, src, lengths):
        # src: [B, T], lengths: [B]
        embedded = self.embedding(src)  # [B, T, E]
        packed = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, hidden = self.rnn(packed)
        outputs, _ = pad_packed_sequence(packed_out, batch_first=True)  # [B, T, H*dirs]

        # If bidirectional, we need to process hidden state properly
        if self.bidirectional:
            if self.cell_type == 'LSTM':
                # For LSTM we have both hidden and cell states
                h_n, c_n = hidden
                # Combine forward and backward states by averaging
                h_n = torch.add(h_n[0:self.layers], h_n[self.layers:]) / 2
                c_n = torch.add(c_n[0:self.layers], c_n[self.layers:]) / 2
                hidden = (h_n, c_n)
            else:
                # For GRU/RNN we only have hidden state
                hidden = torch.add(hidden[0:self.layers], hidden[self.layers:]) / 2

        return outputs, hidden

In [8]:
# Decoder
class Decoder(nn.Module):
    """Decoder with attention support"""
    def __init__(self, vocab_size, emb_size, enc_hid, dec_hid,
                 layers=1, cell="LSTM", dropout=0.0, use_attn=True):
        super().__init__()
        self.use_attn = use_attn
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.cell_type = cell

        # ----- dimensions depend on whether we concatenate context -----
        if use_attn:
            self.attention = BahdanauAttention(enc_hid, dec_hid)
            rnn_input_dim = emb_size + enc_hid            # [E ⊕ Henc]
            fc_input_dim  = dec_hid + enc_hid + emb_size  # [Hdec ⊕ Henc ⊕ E]
        else:
            rnn_input_dim = emb_size
            fc_input_dim  = dec_hid + emb_size

        rnn_cls = {"LSTM": nn.LSTM, "GRU": nn.GRU, "RNN": nn.RNN}[cell]
        self.rnn = rnn_cls(rnn_input_dim, dec_hid,
                           num_layers=layers,
                           dropout=dropout if layers > 1 else 0.0,
                           batch_first=True)
        self.fc = nn.Linear(fc_input_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, encoder_outputs, mask):
        """Forward pass through the decoder"""
        emb = self.embedding(input_token).unsqueeze(1)     # [B,1,E]
        emb = self.dropout(emb)

        if self.use_attn:
            # ---- additive attention ----
            if self.cell_type == 'LSTM':
                dec_h = hidden[0][-1]
            else:
                dec_h = hidden[-1]

            attn_w = self.attention(dec_h, encoder_outputs, mask)          # [B,Tenc]
            ctx    = torch.bmm(attn_w.unsqueeze(1), encoder_outputs)        # [B,1,Henc]
            rnn_in = torch.cat((emb, ctx), dim=2)                           # [B,1,E+Henc]
        else:
            ctx = None
            attn_w = None
            rnn_in = emb                                                    # [B,1,E]

        out, hidden = self.rnn(rnn_in, hidden)       # [B,1,Hdec]
        out = out.squeeze(1)                         # [B,Hdec]
        emb = emb.squeeze(1)                         # [B,E]

        if self.use_attn:
            ctx = ctx.squeeze(1)                     # [B,Henc]
            logits = self.fc(torch.cat((out, ctx, emb), dim=1))
        else:
            logits = self.fc(torch.cat((out, emb), dim=1))

        return F.log_softmax(logits, dim=1), hidden, attn_w

In [9]:
# Sequence-to-Sequence model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx, device='cpu'):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
        self.device = device

    def forward(self, src, src_lens, tgt, teacher_forcing_ratio=0.5):
        """Forward pass with teacher forcing ratio control"""
        enc_out, hidden = self.encoder(src, src_lens)
        mask = (src != self.pad_idx)
        B, T = tgt.size()
        outputs = torch.zeros(B, T-1, self.decoder.fc.out_features, device=self.device)
        input_tok = tgt[:, 0]  # <sos>

        for t in range(1, T):
            out, hidden, _ = self.decoder(input_tok, hidden, enc_out, mask)
            outputs[:, t-1] = out

            # Teacher forcing: with probability, use ground truth as next input
            # Otherwise use predicted token
            teacher_force = random.random() < teacher_forcing_ratio
            if teacher_force:
                input_tok = tgt[:, t]
            else:
                input_tok = out.argmax(1)

        return outputs

    def infer_greedy(self, src, src_lens, tgt_vocab, max_len=50):
        """Greedy inference/generation"""
        enc_out, hidden = self.encoder(src, src_lens)
        mask = (src != self.pad_idx)
        B = src.size(0)
        input_tok = torch.full((B,), tgt_vocab.sos_idx, device=self.device, dtype=torch.long)
        generated = []

        for _ in range(max_len):
            out, hidden, _ = self.decoder(input_tok, hidden, enc_out, mask)
            input_tok = out.argmax(1)
            generated.append(input_tok.unsqueeze(1))
            if (input_tok == tgt_vocab.eos_idx).all():
                break

        return torch.cat(generated, dim=1)

## 5. Training and Evaluation Functions

In [10]:
# Utility functions
def seed_everything(seed=42):
    """Set seeds for reproducibility"""
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [11]:
# Evaluation and prediction functions
def compute_accuracy(model, loader, tgt_vocab, src_vocab, device):
    """Compute accuracy on a dataset"""
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for src, src_lens, tgt in loader:
            src, src_lens, tgt = (x.to(device) for x in (src, src_lens, tgt))
            pred = model.infer_greedy(src, src_lens, tgt_vocab, max_len=tgt.size(1))

            # iterate over the batch
            for b in range(src.size(0)):
                # Convert indices to strings
                pred_str = tgt_vocab.decode(pred[b].cpu().tolist())
                gold_str = tgt_vocab.decode(tgt[b, 1:].cpu().tolist())  # skip <sos>

                # Check if prediction is correct
                correct += (pred_str == gold_str)
                total += 1

    accuracy = correct / total if total else 0.0
    return accuracy

def predict_examples(model, loaders, src_vocab, tgt_vocab, device, num_examples=5):
    """Generate predictions for a few examples from the validation set"""
    model.eval()

    # Get some validation examples
    val_loader = loaders['val']
    val_iter = iter(val_loader)
    srcs, src_lens, tgts = next(val_iter)

    # Make predictions
    srcs, src_lens, tgts = srcs.to(device), src_lens.to(device), tgts.to(device)
    outputs = model.infer_greedy(srcs, src_lens, tgt_vocab, max_len=tgts.size(1))

    # Print results
    print("Example predictions:")
    print("-" * 50)
    for i in range(min(num_examples, srcs.size(0))):
        src_text = src_vocab.decode(srcs[i].cpu().tolist(), strip_specials=True)
        tgt_text = tgt_vocab.decode(tgts[i, 1:].cpu().tolist(), strip_specials=True)  # Skip <sos>
        pred_text = tgt_vocab.decode(outputs[i].cpu().tolist(), strip_specials=True)

        print(f"Source: {src_text}")
        print(f"Target: {tgt_text}")
        print(f"Prediction: {pred_text}")
        print("-" * 50)

def visualize_attention(model, src_text, src_vocab, tgt_vocab, device):
    """Visualize attention weights for a source text"""
    model.eval()

    # Prepare input
    src_indices = src_vocab.encode(src_text, add_sos=True, add_eos=True)
    src_tensor = torch.tensor([src_indices], dtype=torch.long).to(device)
    src_lens = torch.tensor([len(src_indices)], dtype=torch.long).to(device)

    # Get encoder outputs
    enc_outputs, hidden = model.encoder(src_tensor, src_lens)
    mask = (src_tensor != src_vocab.pad_idx)

    # Initialize decoding
    input_tok = torch.tensor([tgt_vocab.sos_idx], device=device)
    generated = []
    attention_weights = []

    # Generate output and collect attention weights
    for _ in range(50):  # Max length
        out, hidden, attn_w = model.decoder(input_tok, hidden, enc_outputs, mask)
        if attn_w is not None:
            attention_weights.append(attn_w.squeeze().cpu().detach().numpy())

        input_tok = out.argmax(1)
        generated.append(input_tok.item())

        if input_tok.item() == tgt_vocab.eos_idx:
            break

    # Convert to text
    prediction = tgt_vocab.decode(generated, strip_specials=True)

    return prediction, attention_weights

In [12]:
# Training function
def train_model(
    model,
    loaders,
    src_vocab,
    tgt_vocab,
    device,
    config,
    save_path=None,
    log_to_wandb=True
):
    """Train a sequence-to-sequence model"""
    criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.pad_idx)

    # Select optimizer based on config
    if config.optimizer.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config.lr)
    elif config.optimizer.lower() == 'nadam':
        optimizer = optim.NAdam(model.parameters(), lr=config.lr)
    else:
        optimizer = optim.Adam(model.parameters(), lr=config.lr)

    # Track best validation accuracy
    best_val_acc = 0.0

    # Main training loop
    for epoch in range(1, config.epochs + 1):
        model.train()
        total_loss = 0.0

        # Training batches
        for src, src_lens, tgt in loaders['train']:
            src, src_lens, tgt = src.to(device), src_lens.to(device), tgt.to(device)

            optimizer.zero_grad()
            output = model(src, src_lens, tgt, teacher_forcing_ratio=config.teacher_forcing)
            loss = criterion(output.reshape(-1, output.size(-1)), tgt[:,1:].reshape(-1))
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()

        train_loss = total_loss / len(loaders['train'])

        # Validation loss
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for src, src_lens, tgt in loaders['val']:
                src, src_lens, tgt = src.to(device), src_lens.to(device), tgt.to(device)
                output = model(src, src_lens, tgt, teacher_forcing_ratio=0.0)  # No teacher forcing during validation
                val_loss += criterion(output.reshape(-1, output.size(-1)),
                                    tgt[:,1:].reshape(-1)).item()
        val_loss /= len(loaders['val'])

        # Compute accuracy metrics
        train_acc = compute_accuracy(model, loaders['train'], tgt_vocab, src_vocab, device)
        val_acc = compute_accuracy(model, loaders['val'], tgt_vocab, src_vocab, device)

        # Save model if it's the best so far
        if val_acc > best_val_acc and save_path:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f"Saved new best model with validation accuracy: {val_acc:.4f}")

        # Log metrics
        print(f"Epoch {epoch}/{config.epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        if log_to_wandb:
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'validation_loss': val_loss,
                'train_accuracy': train_acc,
                'validation_accuracy': val_acc
            })

    # Final evaluation on test set
    if 'test' in loaders:
        test_acc = compute_accuracy(model, loaders['test'], tgt_vocab, src_vocab, device)
        print(f"Final test accuracy: {test_acc:.4f}")

        if log_to_wandb:
            wandb.log({'test_accuracy': test_acc})
    else:
        test_acc = None

    return model, test_acc

## 6. Main Execution

In [13]:
def main():
    """Main function to train a model using only wandb sweep configuration"""
    # Initialize wandb
    run = wandb.init()

    # Get the config from wandb
    config = run.config

    # Map 'cell' to 'cell_type' for compatibility
    if 'cell' in config:
        config.cell_type = config.cell

    # Set default value for dec_layers (same as enc_layers)
    config.dec_layers = config.enc_layers if hasattr(config, 'enc_layers') else 1

    # Set default value for use_attention (True)
    config.use_attention = False

    # Set default language and dataset format
    config.language = 'te'  # Telugu language
    config.dataset_format = 'dakshina'

    # Set seeds for reproducibility
    seed_everything(config.seed)

    # Download and setup Dakshina dataset
    dakshina_path = download_dakshina_dataset()
    print(f"Using Dakshina dataset from: {dakshina_path}")

    # Load data
    loaders, src_vocab, tgt_vocab = get_dataloaders(
        language=config.language,
        dataset_format=config.dataset_format,
        batch_size=config.batch_size,
        device=device
    )

    print(f"Source vocabulary size: {src_vocab.size}")
    print(f"Target vocabulary size: {tgt_vocab.size}")

    # Create model
    encoder = Encoder(
        src_vocab.size, config.emb_size, config.hidden_size,
        layers=config.enc_layers, cell=config.cell_type,
        dropout=config.dropout, bidirectional=config.bidirectional
    ).to(device)

    # Calculate encoder output dimension (doubled if bidirectional)
    enc_out_dim = config.hidden_size * 2 if config.bidirectional else config.hidden_size

    decoder = Decoder(
        tgt_vocab.size, config.emb_size, enc_out_dim, config.hidden_size,
        layers=config.dec_layers, cell=config.cell_type,
        dropout=config.dropout, use_attn=config.use_attention
    ).to(device)

    model = Seq2Seq(encoder, decoder, src_vocab.pad_idx, device=device).to(device)

    # Train model
    model, test_acc = train_model(
        model=model,
        loaders=loaders,
        src_vocab=src_vocab,
        tgt_vocab=tgt_vocab,
        device=device,
        config=config,
        save_path="best_model.pt",
        log_to_wandb=True
    )

    # Save final model
    torch.save(model.state_dict(), "final_model.pt")

    # Generate some predictions
    predict_examples(model, loaders, src_vocab, tgt_vocab, device, num_examples=5)

    return model, src_vocab, tgt_vocab, loaders

In [14]:
# Run the main function without WandB for simplicity
# model, src_vocab, tgt_vocab, loaders = main()

In [ ]:
# Initialize wandb
wandb.login()

# Use the exact sweep configuration provided
sweep_cfg = {
    'method': 'bayes',  # Use Bayesian optimization
    'name': 'test_run',
    'metric': {'name': 'validation_accuracy', 'goal': 'maximize'},
    'parameters': {
        # Model architecture
        'emb_size': {'values': [128, 256, 512]},
        'hidden_size': {'values': [128, 256, 512, 1024]},
        'enc_layers': {'values': [1, 2, 3, 4]},
        'cell': {'values': ['RNN', 'GRU', 'LSTM']},
        'bidirectional': {'values': [True, False]},  # Bidirectional encode

        # Training parameters
        'dropout': {'values': [0.0, 0.1, 0.2, 0.3, 0.5]},
        'lr': {'values': [1e-4, 2e-4, 5e-4, 8e-4, 1e-3]},
        'batch_size': {'values': [32, 64, 128]},
        'epochs': {'values': [10, 15, 20]},
        'teacher_forcing': {'values': [0.3, 0.5, 0.7, 1.0]},  # Explicit teacher forcing
        'optimizer': {'values': ['Adam', 'NAdam']},  # Added optimizer options
        # Reproducibility
        'seed': {'values': [42, 43, 44, 45, 46]},  # Different seeds for robustness
    }
}

# Start the sweep
sweep_id = wandb.sweep(sweep_cfg, project="DA6401_Assignment_3")
wandb.agent(sweep_id, function=main, count=1)  # Run 20 trials (adjust as needed)

<IPython.core.display.Javascript object>